# Assignment 1 - Group 2

### Exercise 1

**a) In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations.**

What we observed is that maximal reaction activities were generally not equal. The values in the pathway vary significantly. This happens beacuse the maximal activity is the maximum rate a reaction can have. This is determined by the concentration of the enzymes that catalyses the reactions. This is deifferent from what we saw with a linear pathway where all reactions must be equal to maintain the mass balance.

**b) Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions.**

- Transport reactions: are not directly catalyzed by a single enzyme or gene product. Depend on other properties like concentration gradients which makes gene expression-derived data irrelevant.

- Non-enzymatic reactions: occur randomly without an enzyme. Depend on chemical properties of the involved metabolites so we have no genes. An example can be EX_fum_e.



### Exercise 2

In [2]:
!pip install cobra
import cobra
import pandas as pd

  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached optlang-1.8.3-py2.py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 1.0/1.2 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.6 MB/s eta 0:00:00
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
   ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.0 MB 2.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.0/6.0 MB 2.8 MB/s eta 0:00:02
   ------------- -------------------------- 2.1/6.0 MB 3.5 MB/s eta 0:00:02
   -------------------- ------------------- 3.1/6.0 MB 3.7 MB/s eta 0:00:01
   --------------------------- ------------ 4.2/6.0 MB 3.9 MB/s

In [3]:
model = cobra.io.load_json_model('e_coli_core.json')
df = pd.read_csv('e_coli_core_expression.csv')

In [4]:
mapping = dict(zip(df["# Reaction ID"], df[" reaction activity [mmol/gDW/h] "]))

In [5]:
#Reversible/Irreversible reactions, lower and upper bounds
DEFAULT_HIGH = 1000.0
changed = []
unchanged = []

for reaction in model.reactions:
    #glucose exchange reaction
    if reaction.id == "EX_glc__D_e":
        reaction.lower_bound = -DEFAULT_HIGH
        reaction.upper_bound =  DEFAULT_HIGH
        changed.append((reaction.id, "special: EX_glc__D_e set to ±1000"))
        continue

    #ATP maintenance
    if reaction.id == "ATPM":
        if reaction.id in mapping:
            val = mapping[reaction.id]
            reaction.upper_bound = val
            changed.append((reaction.id, f"ATPM: lower preserved, upper set to {val}"))
        else:
            unchanged.append(reaction.id)
        continue

    if reaction.id in mapping:
        val = float(mapping[reaction.id])
        if reaction.lower_bound < 0:
            #reversible reaction
            reaction.lower_bound = -val
            reaction.upper_bound =  val
            changed.append((reaction.id, f"reversible set to ±{val}"))
        else:
            #irreversible reaction
            reaction.lower_bound = 0.0
            reaction.upper_bound =  val
            changed.append((reaction.id, f"irreversible set to [0, {val}]"))
    else:
        #no data
        unchanged.append(reaction.id)

In [6]:
rows = []
for reaction in model.reactions:
    rows.append((reaction.id, reaction.lower_bound, reaction.upper_bound))

for r in rows[:20]:
    print(f"{r[0]}\t{r[1]}\t{r[2]}")

PFK	0.0	12.12
PFL	0.0	1.0
PGI	-13.12	13.12
PGK	-23.13	23.13
PGL	0.0	8.12
ACALD	-1.16	1.16
AKGt2r	-3.1	3.1
PGM	-20.01	20.01
PIt2r	-6.03	6.03
ALCD2x	-9.01	9.01
ACALDt	-2.29	2.29
ACKr	-1.19	1.19
PPC	0.0	2.56
ACONTa	-25.35	25.35
ACONTb	-25.35	25.35
ATPM	8.39	1000.0
PPCK	0.0	25.23
ACt2r	-3.23	3.23
PPS	0.0	2.5
ADK1	-30.57	30.57


### Exercise 3

**a) Carry out an FVA for all reactions in the model and print out the resulting minimal and maximal fluxes
per reaction (simple printout sufficient).**

In [8]:
from cobra.flux_analysis import flux_variability_analysis

FVA_results = flux_variability_analysis(model=model, reaction_list=model.reactions, loopless=False, fraction_of_optimum=0.0)

print("Reaction ID\tMin Flux\tMax Flux")
for rxn_id, row in FVA_results.iterrows():
    print(f"{rxn_id}\t{row['minimum']}\t{row['maximum']}")


Reaction ID	Min Flux	Max Flux
PFK	0.3346774193548387	11.153555525746384
PFL	0.0	1.0
PGI	-0.08532258064516074	9.020350656890786
PGK	-17.596001612302814	-0.879354838709676
PGL	0.0	4.522387640638964
ACALD	-1.16	0.0
AKGt2r	-3.1	0.0
PGM	-17.038932281976656	-0.8793548387096765
PIt2r	0.0	1.7830772865042592
ALCD2x	-1.16	0.0
ACALDt	-1.16	0.0
ACKr	-1.19	0.0
PPC	0.0	2.56
ACONTa	-2.05779213860328e-16	5.775555555555558
ACONTb	0.0	5.775555555555557
ATPM	8.39	39.983750000000015
PPCK	0.0	4.690000000000002
ACt2r	-1.19	0.0
PPS	0.0	2.5
ADK1	0.0	2.5
AKGDH	0.0	4.126039421977849
ATPS4r	0.42666666666666864	27.640999999999995
PTAr	2.5743667056490966e-16	1.19
PYK	0.0	9.232
BIOMASS_Ecoli_core_w_GAM	0.0	0.48470309797054584
PYRt2	-1.26	0.0
CO2t	-13.674766918501094	1.61533333333333
RPE	-0.34620184843504465	2.6885712014932817
CS	4.11558427720656e-16	5.775555555555558
RPI	-1.833816439145686	0.0
SUCCt2_2	0.0	2.36
CYTBD	0.9999999999999949	24.51200000000001
D_LACt2	-2.0700000000000003	0.0
ENO	0.8793548387096786	17.0389

**b) Identify any reactions with gene expression-imposed maximal reaction activities, whose permissible
flux range in forward direction is nonzero yet comes out less than its upper flux bound. State their number and explain why a set of reactions behaves this way. (You can ignore reaction FORt here which as seen
in the practical has a nonconventional direction definition.)**

In [ ]:
violations = []

for rxn_id, row in FVA_results.iterrows():
    if rxn_id == "FORt":
        continue
    if rxn_id in mapping: 
        rxn = model.reactions.get_by_id(rxn_id)
        if row["maximum"] > 0 and row["maximum"] < rxn.upper_bound:
            violations.append((rxn_id, row["maximum"], rxn.upper_bound))

print(f"# reactions meeting criterion: {len(violations)}")
print("Reaction ID\tFVA max\tUpper bound")
for rxn_id, fva_max, ub in violations:
    print(f"{rxn_id}\t{fva_max}\t{ub}")

# reactions meeting criterion: 36
Reaction ID	FVA max	Upper bound
PFK	11.153555525746384	12.12
PGI	9.020350656890786	13.12
PGL	4.522387640638964	8.12
PIt2r	1.7830772865042592	6.03
ACONTa	5.775555555555558	25.35
ACONTb	5.775555555555557	25.35
PPCK	4.690000000000002	25.23
ADK1	2.5	30.57
AKGDH	4.126039421977849	24.35
ATPS4r	27.640999999999995	60.5
PTAr	1.19	4.47
PYK	9.232	26.78
CO2t	1.61533333333333	30.45
RPE	2.6885712014932817	5.67
CS	5.775555555555558	20.56
CYTBD	24.51200000000001	40.56
ENO	17.038932281976656	28.78
SUCDi	24.322000000000003	24.34
TALA	1.4262372946170052	4.45
TKT1	1.4262372946170054	3.34
TKT2	1.2623339068762753	3.34
TPI	8.803555525746381	45.56
FBA	8.803555525746381	24.45
FUM	4.252000000000002	24.35
G6PDH2r	4.52238764063897	5.68
GAPD	17.596001612302814	26.13
GLCpts	9.493299814114787	16.34
GLUDy	3.45	7.35
GLUN	3.45	6.25
GLUSy	3.4500000000000006	7.35
GND	4.522387640638969	6.72
ICDHyr	5.20213051673949	15.06
ICL	4.252	14.07
MALS	4.252000000000002	9.12
O2t	12.256000000000002	40

Some reactions with expression-based upper bounds show a smaller FVA maximum than that bound, because other reactions in the network restrict the flux. Substrates may be limited, products may not be demanded, or mass balance with neighboring reactions caps the feasible flow. So even though the individual bound is higher, the network as a whole prevents the reaction from reaching it.

**c) How many reactions have a positive minimal flux in the FVA? State their number and explain why a
set of reactions behaves this way.**

In [10]:
positive_min = []

for rxn_id, row in FVA_results.iterrows():
    if row["minimum"] > 0:
        positive_min.append((rxn_id, row["minimum"], row["maximum"]))

print(f"# reactions with positive minimal flux: {len(positive_min)}")
print("Reaction ID\tMin Flux\tMax Flux")
for rxn_id, vmin, vmax in positive_min:
    print(f"{rxn_id}\t{vmin}\t{vmax}")


# reactions with positive minimal flux: 16
Reaction ID	Min Flux	Max Flux
PFK	0.3346774193548387	11.153555525746384
ATPM	8.39	39.983750000000015
ATPS4r	0.42666666666666864	27.640999999999995
PTAr	2.5743667056490966e-16	1.19
CS	4.11558427720656e-16	5.775555555555558
CYTBD	0.9999999999999949	24.51200000000001
ENO	0.8793548387096786	17.038932281976656
TPI	0.33467741935483825	8.803555525746381
EX_h_e	4.7346351231705704e-14	21.802165750892723
EX_h2o_e	0.5379999999999973	18.59087349355403
FBA	0.3346774193548389	8.803555525746381
GAPD	0.8793548387096771	17.596001612302814
GLCpts	0.47942857142857154	9.493299814114787
ICDHyr	3.987053314654254e-16	5.20213051673949
NADH16	1.831111111111111	20.26
O2t	0.49999999999999933	12.256000000000002


Some reactions always have a positive minimal flux, because they are essential for the cell. They either have a required lower bound (like ATP maintenance) or the network forces them to run to keep balances and produce key compounds.

### Exercise 4

**a) Carry out an FBA optimization of biomass and report the maximal biomass production rate in the
presence of the implemented constraints.**

In [ ]:
print("Maximal biomass production rate:", model.optimize().objective_value) 
#all flux constraints alr applied in the current state of the model

Maximal biomass production rate: 0.48470309797054517


**b) Determine the “bottleneck(s)” – i. e. those reaction(s) whose flux reaches its maximum.**

In [14]:
solution = model.optimize()

bottlenecks = []

for reaction in model.reactions:
    flux = solution.fluxes[reaction.id]
    if flux == reaction.upper_bound:
        bottlenecks.append((reaction.id, flux, reaction.upper_bound))

print("Bottleneck reactions:")
for rxn_id, flux, ub in bottlenecks:
    print(f"{rxn_id}\tflux={flux}\tupper bound={ub}")


Bottleneck reactions:
PFL	flux=1.0	upper bound=1.0
THD2	flux=4.5	upper bound=4.5
NADH16	flux=20.26	upper bound=20.26


**c) The model does not use (have non-zero flux for) all reactions in the model constrained with maximal
reaction activities. Pick a reaction with maximal reaction activity constraints and zero flux in the optimal
solution, and explain from the network context why it carries not carry flux. (Hint: Use the online demo again.)**

In [16]:
solution = model.optimize()

unused_constrained = []

for rxn_id in mapping.keys():
    flux = solution.fluxes[rxn_id]
    if abs(flux) == 0:
        unused_constrained.append(rxn_id)

print("Constrained reactions with zero flux in optimum:")
for rxn_id in unused_constrained:
    print(rxn_id)


Constrained reactions with zero flux in optimum:
AKGt2r
ACALDt
PPCK
PPS
ADK1
SUCCt2_2
SUCCt3
FBP
FORt2
FRD7
FRUpts2
FUMt2_2
GLNabc
GLUN
GLUSy
GLUt2r
ICL
MALS
MALt2_2
ME1
ME2
NADTRHD


AKGt2r has zero flux because the model does not need to transport α-ketoglutarate. It is already made inside the cell and there is no uptake from outside, so the transporter is unused.